##Setting-Up

In [74]:
!pip install beautifulsoup4
!pip install lxml html5lib requests

from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime
from datetime import time, date, timedelta, datetime

##Getting Google-search URL
####(for Requests and BS4 later)

In [0]:
#basic part of google search
def basic():
  return 'https://www.google.co.in/search?'

In [0]:
#To add + between whitespaces
def get_string(search_string):
  
  value = search_string.split(' ')
  string = ''
  for word in value:
    if len(string)==0: string = word
    else: string = string + '+'+str(word)
  
  return string

In [0]:
#Q parameter ie. query in google search
def q_parameter():
  base = 'q='+get_string(search_string)
  return base

In [0]:
#Detects if there is any quotes ("") or not
def quotes_or_not(search_string):
  value = search_string.split('"')[0]
  
  return_string = ''
  
  if value != search_string: 
    return_string = '&oq='
    return_string += get_string(search_string)
    
  return return_string

In [0]:
#Choose a function:
def function_choose():
  
  print('Choose a function:\n 1. Custom Date-Range -> 1\n 2. Common date functions -> 2\n 3. Verbatim -> 3\n 4. None -> 0')

  valid_values = [0,1,2,3]
  while True:
    try:
      func_num = int(input('Please enter one/two/three: '))
      if func_num not in valid_values: raise ValueError
    except ValueError:
      print('Invalid Value! Please enter only one two or three!')
    else: break
  
  return func_num

In [0]:
#Prompt input till user enters valid date
def get_valid_dates():
  while True:
    try:
      start_date_day = input('Enter day of starting date: ')
      start_date_month = input('Enter month of starting date: ')
      start_date_year = input('Enter year of starting date: ')
      start_date = datetime(int(start_date_year),int(start_date_month),int(start_date_day))
    except ValueError:
      print('Invalid starting Date!')
    else: break
  
  while True:
    try:
      end_date_day = input('Enter day of ending date: ')
      end_date_month = input('Enter month of ending date: ')
      end_date_year = input('Enter year of ending date: ')
      end_date = datetime(int(end_date_year),int(end_date_month),int(end_date_day))
    except ValueError:
      print('Invalid ending Date!')
    else: break
  #When start date comes after end-date  
  if start_date-end_date > timedelta(0):
    temp1 = start_date
    start_date = end_date
    end_date = temp1
    print('Swapping start date and end date, as end date came before start date!')
  return start_date,end_date

In [0]:
#Function to add string to URL in case of custom range
def custom_range(start_date,end_date):
  base_string = '%2Ccd_min%3A'+str(start_date.month)+'%2F'+str(start_date.day)+'%2F'+str(start_date.year)
  
  base_string+= '%2Ccd_max%3A'+str(end_date.month)+'%2F'+str(end_date.day)+'%2F'+str(end_date.year)
  
  return base_string

In [0]:
#Function to add string to URL in case of common past function
def get_common_string():
  value1 = ['h','d','w','m','y']
  
  while True:
    print('#Function Lists:\n--------------------------------------\n')
    print("1. Day-based functions(past day etc.) -> 'd'")
    print("2. Week-based functions(past week etc.) -> 'w'")
    print("3. Month-based functions(past month etc.) -> 'm'")
    print("4. Year-based functions(past year etc.) -> 'y'")
    print("5. Hour-based functions(past hour etc.) -> 'h'")
    try:
      
      value1_in = input('Enter d/m/w/y: ').lower()
      
      if value1_in not in value1: raise ValueError
      
    except ValueError:
      print('Please enter only d, w, m or y! (Caps allowed)')
      
    else: break
        
  value2 = [0,1]
  while True:
    try:
      print('\n#Function Lists:\n--------------------------------------\n')
      print("1. For simple functions(past day, past hour etc.) -> 0")
      print("2. For custom functions(past 4 days, 3 months etc.) -> 1")
      
      value2_in = int(input('Enter 0/1: '))
      
      if value2_in not in value2: raise ValueError
      
    except ValueError:
      print('Please enter only 0 or 1!')
    else: break
  
  integer= ''
  if value2_in == 1:
    while True:
      try:
        integer = str(int(input('No. of days/hours/ ... etc.,(eg. for 5 months enter 5) : ')))
      except ValueError:
        print('Invalid value entered.')
      else: break
  
  return str(value1_in+integer)

In [0]:
#Completely forms the URL, using string addition
def get_URL(search_string):
  
  base = basic()+q_parameter()+quotes_or_not(search_string)
  xtra_string = ''
  flag = function_choose()
  
  if flag ==1:
    start_date, end_date = get_valid_dates()
    custom_range_string = 'cdr%3A1'+custom_range(start_date,end_date)
    xtra_string += custom_range_string
  
  if flag ==2:
    common_function_string = 'qdr:'+get_common_string()
    xtra_string += common_function_string
    
  if flag == 3:
    verbatim_string = 'li:1'
    xtra_string += verbatim_string
  
  if len(xtra_string) !=0:
    xtra_string = '&tbs='+ xtra_string
    
  base += xtra_string
  
  return base

###User-input

In [99]:
#User input
search_string = input()
URL = get_URL(search_string)

Narendra Modi
Choose a function:
 1. Custom Date-Range -> 1
 2. Common date functions -> 2
 3. Verbatim -> 3
 4. None -> 0
Please enter one/two/three: 2
#Function Lists:
--------------------------------------

1. Day-based functions(past day etc.) -> 'd'
2. Week-based functions(past week etc.) -> 'w'
3. Month-based functions(past month etc.) -> 'm'
4. Year-based functions(past year etc.) -> 'y'
5. Hour-based functions(past hour etc.) -> 'h'
Enter d/m/w/y: y

#Function Lists:
--------------------------------------

1. For simple functions(past day, past hour etc.) -> 0
2. For custom functions(past 4 days, 3 months etc.) -> 1
Enter 0/1: 0


## Requests + BeautifulSoup BS4 operations

In [0]:
#uses BS4 functions to extract valuable data
def html_parse(URL, keyword):  
  source = requests.get(URL) 
  soup = BeautifulSoup(source.text,'html.parser')

  all_results = soup.find_all('div', attrs={'class': 'g'})#For each part/article on search
  
  rank =1 #Search Rank
  
  #For CSV
  title_val = []
  keyword_val = []
  rank_val = []
  description_val = []
  link_val = []
  
  #Appending results to arrays
  for result in all_results:
    title = result.find('h3')
    link = result.find('a',href=True)
    
    description = result.find('span', attrs={'class': 'st'})
    
    if link and title:
      link = link['href']
      title = title.get_text()
      if description: description = description.get_text()
      if link != '#':
        title_val.append(title)
        keyword_val.append(keyword)
        rank_val.append(rank)
        description_val.append(description)
        link_val.append(link)
        rank += 1
  #Forming dataframe to for proper format
  found_results = pd.DataFrame({'Title': title_val,
                                'Rank': rank_val,
                                'Link': link_val,
                                'Keyword': keyword_val,
                                'Description': description_val
                               })
  return found_results
    


In [0]:
full_data = html_parse(URL,search_string)

In [0]:
#Create and write CSV file
name = 'Headlines'+' of '+'_'+search_string+'_'+'.csv'
out = full_data.to_csv(name)

In [0]:
#Download File from colab
from google.colab import files
files.download(name)